In [ ]:
import os
import glob
import requests
import json
import pandas as pd
import csv

In [ ]:
seed_path = os.path.join('data', 'seed')
top_songs_filename = os.path.join('data', 'raw', 'top_songs_with_lyrics.csv')

# Read data

In [ ]:
all_files = glob.glob(seed_path + "/*.csv")
song_data = pd.concat(pd.read_csv(f) for f in all_files)
song_data.head(60)

In [ ]:
# Normalize artist and title and drop duplicate rows
song_data['artists'] = song_data['artists'].str.upper() 
song_data['title'] = song_data['title'].str.upper() 
song_data.drop_duplicates(subset=['title', 'artists'], keep='first', inplace=True)

song_data.head()

# Get lyrics from api

In [ ]:
# TODO: Ignorar las canciones que no tienen lyrics
# Referencia de como usar el api de music match https://youtu.be/WFRzKmpepj4
base_url= 'https://api.musixmatch.com/ws/1.1/'
api_key = "3950aca40989ee9f7d05723278e3fdfd"

In [ ]:
def get_lyric(row):
    title = row['title']
    artist = row['artists']
    lyric_matcher = 'matcher.lyrics.get?format=json&callback=callback'
    url = f"{base_url}/{lyric_matcher}&q_track={title}&q_artist={artist}&apikey={api_key}"
    # print(title, artist, url)
    
    response = requests.get(url)
    data = response.json() # lee la información en json (cuidado con jsonp, se tiene que hacer mayor procesamiento)
    if data['message']['header']['status_code'] == 200:
        lyric = data['message']['body']['lyrics']['lyrics_body'].replace('******* This Lyrics is NOT for Commercial use *******', '')
        # print(lyric)
        
        row['lyric'] = lyric.replace('(1409620732275)',' ')
        row['explicit'] = data['message']['body']['lyrics']['explicit']
    return row

def get_music_genre(row):
    title = row['title']
    artist = row['artists']
    url = f"{base_url}/matcher.track.get?format=json&callback=callback&q_track={title}&q_artist={artist}&apikey={api_key}"
    # print(title, artist, url)
    
    response = requests.get(url)
    data = response.json()
    # print(data)
    if data['message']['header']['status_code'] == 200:
        track = data['message']['body']['track']
        # print(track)
        
        row['musixmatch_rating'] = track['track_rating']
        # row['artist_name'] = track['artist_name']
        # row['track_name'] = track['track_name']

        if len(track['primary_genres']['music_genre_list']) > 0:
            row['genre'] = track['primary_genres']['music_genre_list'][0]['music_genre']['music_genre_name_extended']
    return row

In [ ]:
song_data = song_data.apply(get_lyric, axis=1)
song_data = song_data.apply(get_music_genre, axis=1)

In [ ]:
song_data.head(50)

# Export lyrics data

In [ ]:
song_data.to_csv(
    top_songs_filename, 
    index=False, 
    quoting=csv.QUOTE_NONNUMERIC  # Prevent errors on strings with , and \n characters
)